# Prediction of energy market prices 

You are given meteorological and market data for part of 2025. [Meteorological data file](https://github.com/jkved/enerheads-quant-challenge/blob/main/data/weather_location_Vilnius.csv)(s) contains day-ahead and intraday forecasts for meteorological variables for a single location. The data description is available on [OpenMeteo](https://open-meteo.com/en/docs) docs and the timezone here is in UTC. This will contain some of your predictors values. Day ahead value (with suffix `previous_day1`) is known 24 hours before delivery time, intraday (no suffix column) is around 1 hour before or at delivery time. Market data (predictors and target values) is given in file [market data file](https://github.com/jkved/enerheads-quant-challenge/blob/main/data/market_data.csv), the data is also publicly available on [Baltic transparency dashboard](https://baltic.transparency-dashboard.eu/). Here index is in UTC timezone and two columns here are considered our target variables:
- `10YLT-1001A0008Q_DA_eurmwh` - this is Nord Pool day-ahead auction cleared prices (EUR/MWh). It is resolved day before delivery day (day-ahead), i.e. today at 10:00 UTC we find out prices for tommorrow CET day (22:00 UTC today -> 22:00 UTC tommorrow). Only weather data with `previous_day1` suffix is available at inference time
- `LT_up_sa_cbmp` or `LT_down_sa_cbmp` - this is mFRR activation prices. Generally only up or down activations take place at the same time so price is duplicated in these columns. It is resolved at delivery time (intraday). All weather data and Nord Pool prices are available at inference time but all other market data is visible with a 30 minute lag, i.e. for an mFRR activation price @ 11:00, all other market data is visible only up to (not including) 10:30.

Complete the following tasks:

1. Create Nord Pool prices forecasting model in day-ahead setting.
2. Create mFRR prices forecasting model in intraday setting
3. Implement certain evaluation metrics for prices:
   - you wish to accurately guess times when smallest and largest prices of the day take shape.
   - you wish to know how many instances there are with spreads between smallest and largest prices being bigger than X (say, 200 EUR/MWh).
4. Choose a collection of 2-3 plots to visualize the performance of both models.

    


# Table of Contents


# Introduction

# Importing the Data

In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
market_df = pd.read_csv("data/market_data.csv", index_col=0)
market_df.index = pd.to_datetime(market_df.index, utc=True)
market_df.head()

,EE_afrr_up_activ,EE_afrr_down_activ,LV_afrr_up_activ,LV_afrr_down_activ,LT_afrr_up_activ,LT_afrr_down_activ,EE_afrr_up_min_bid,EE_afrr_up_max_bid,EE_afrr_down_min_bid,EE_afrr_down_max_bid,...,LT_up_da_cbmp,LT_down_sa_cbmp,LT_down_da_cbmp,EE_dsb,LV_dsb,LT_dsb,EE_imbalance_price,LV_imbalance_price,LT_imbalance_price,10YLT-1001A0008Q_DA_eurmwh
2025-03-01 00:00:00+00:00,1.818,0.018,3.048,0.0,2.05,0.00,400.0,927.0,-273.0,20.0,...,NaN,27.27,NaN,1,-1.0,1.0,400.00,10.19,927.02,120.48
2025-03-01 00:15:00+00:00,1.729,0.000,3.547,0.0,2.40,0.00,400.0,400.0,-273.0,20.0,...,NaN,27.27,NaN,1,-1.0,1.0,400.00,10.19,932.81,120.48
2025-03-01 00:30:00+00:00,1.070,0.000,3.506,0.0,0.38,1.50,400.0,927.0,-273.0,20.0,...,NaN,NaN,NaN,1,-1.0,1.0,400.00,10.19,119.23,120.48
2025-03-01 00:45:00+00:00,0.604,0.221,1.828,0.0,0.00,2.48,400.0,927.0,-273.0,20.0,...,NaN,27.27,NaN,1,-1.0,-1.0,400.00,10.19,-303.44,120.48
2025-03-01 01:00:00+00:00,2.829,0.000,2.324,0.0,0.00,1.75,400.0,927.0,-273.0,20.0,...,NaN,27.27,NaN,1,-1.0,-1.0,560.36,10.19,-291.36,117.15


In [3]:
files = glob.glob("data/weather_location_*.csv")
dfs = []

for file in files:
    city = os.path.splitext(os.path.basename(file))[0].replace("weather_location_", "")
    df = pd.read_csv(file, )
    df["city"] = city
    dfs.append(df)

waether_df = pd.concat(dfs, ignore_index=True)
waether_df.head()

,Unnamed: 0,wind_speed_80m,wind_speed_80m_previous_day1,wind_direction_80m,wind_direction_80m_previous_day1,direct_radiation,direct_radiation_previous_day1,diffuse_radiation,diffuse_radiation_previous_day1,cloud_cover,cloud_cover_previous_day1,temperature_2m,temperature_2m_previous_day1,relative_humidity_2m,relative_humidity_2m_previous_day1,city
0,2025-03-01 00:00:00+00:00,13.333627,13.684512,125.0,87.0,0.0,0.0,0.0,0.0,100.0,100.0,0.8785,0.8285,96.0,96.0,Alytus
1,2025-03-01 01:00:00+00:00,12.280973,14.737166,123.0,99.0,0.0,0.0,0.0,0.0,100.0,100.0,0.7285,0.7285,96.0,97.0,Alytus
2,2025-03-01 02:00:00+00:00,13.333627,12.631857,118.0,101.0,0.0,0.0,0.0,0.0,100.0,100.0,0.7785,0.7285,96.0,96.0,Alytus
3,2025-03-01 03:00:00+00:00,11.228318,9.473893,130.0,102.0,0.0,0.0,0.0,0.0,100.0,100.0,0.7785,0.6285,98.0,97.0,Alytus
4,2025-03-01 04:00:00+00:00,9.123008,7.368583,152.0,87.0,0.0,0.0,0.0,0.0,100.0,100.0,0.7785,0.5285,99.0,97.0,Alytus


In [4]:
waether_df = waether_df.rename(columns={"Unnamed: 0": "time"})
waether_df["time"] = pd.to_datetime(waether_df["time"], utc=True)
waether_df.head()

,time,wind_speed_80m,wind_speed_80m_previous_day1,wind_direction_80m,wind_direction_80m_previous_day1,direct_radiation,direct_radiation_previous_day1,diffuse_radiation,diffuse_radiation_previous_day1,cloud_cover,cloud_cover_previous_day1,temperature_2m,temperature_2m_previous_day1,relative_humidity_2m,relative_humidity_2m_previous_day1,city
0,2025-03-01 00:00:00+00:00,13.333627,13.684512,125.0,87.0,0.0,0.0,0.0,0.0,100.0,100.0,0.8785,0.8285,96.0,96.0,Alytus
1,2025-03-01 01:00:00+00:00,12.280973,14.737166,123.0,99.0,0.0,0.0,0.0,0.0,100.0,100.0,0.7285,0.7285,96.0,97.0,Alytus
2,2025-03-01 02:00:00+00:00,13.333627,12.631857,118.0,101.0,0.0,0.0,0.0,0.0,100.0,100.0,0.7785,0.7285,96.0,96.0,Alytus
3,2025-03-01 03:00:00+00:00,11.228318,9.473893,130.0,102.0,0.0,0.0,0.0,0.0,100.0,100.0,0.7785,0.6285,98.0,97.0,Alytus
4,2025-03-01 04:00:00+00:00,9.123008,7.368583,152.0,87.0,0.0,0.0,0.0,0.0,100.0,100.0,0.7785,0.5285,99.0,97.0,Alytus


Since weather is hourly, I will also resample the market data to hourly

In [7]:
market_hourly = market_df.resample("1h").last().reset_index()
market_hourly = market_hourly.rename(columns={"index": "time"})
market_hourly.head()

,time,EE_afrr_up_activ,EE_afrr_down_activ,LV_afrr_up_activ,LV_afrr_down_activ,LT_afrr_up_activ,LT_afrr_down_activ,EE_afrr_up_min_bid,EE_afrr_up_max_bid,EE_afrr_down_min_bid,...,LT_up_da_cbmp,LT_down_sa_cbmp,LT_down_da_cbmp,EE_dsb,LV_dsb,LT_dsb,EE_imbalance_price,LV_imbalance_price,LT_imbalance_price,10YLT-1001A0008Q_DA_eurmwh
0,2025-03-01 00:00:00+00:00,0.604,0.221,1.828,0.0,0.00,2.48,400.0,927.0,-273.0,...,NaN,27.27,NaN,1,-1.0,-1.0,400.00,10.19,-303.44,120.48
1,2025-03-01 01:00:00+00:00,0.517,0.684,1.854,0.0,0.00,2.42,400.0,927.0,-273.0,...,NaN,27.27,NaN,1,-1.0,-1.0,560.36,10.19,-301.87,117.15
2,2025-03-01 02:00:00+00:00,0.000,0.000,3.462,0.0,2.42,0.00,0.0,0.0,-273.0,...,NaN,27.27,NaN,1,-1.0,-1.0,400.00,10.19,-6.29,112.42
3,2025-03-01 03:00:00+00:00,0.000,0.000,3.734,0.0,0.00,2.40,0.0,0.0,-273.0,...,NaN,145.17,NaN,1,-1.0,-1.0,161.88,183.15,-336.08,109.94
4,2025-03-01 04:00:00+00:00,1.250,0.000,3.488,0.0,2.33,0.00,400.0,400.0,-273.0,...,NaN,145.17,NaN,1,-1.0,1.0,129.90,128.09,928.94,114.68


In [9]:
merged_df = pd.merge(waether_df, market_hourly, on="time", how="inner")
merged_df.sort_values(by="time").head()

,time,wind_speed_80m,wind_speed_80m_previous_day1,wind_direction_80m,wind_direction_80m_previous_day1,direct_radiation,direct_radiation_previous_day1,diffuse_radiation,diffuse_radiation_previous_day1,cloud_cover,...,LT_up_da_cbmp,LT_down_sa_cbmp,LT_down_da_cbmp,EE_dsb,LV_dsb,LT_dsb,EE_imbalance_price,LV_imbalance_price,LT_imbalance_price,10YLT-1001A0008Q_DA_eurmwh
0,2025-03-01 00:00:00+00:00,13.333627,13.684512,125.000000,87.000000,0.0,0.0,0.0,0.0,100.0,...,NaN,27.27,NaN,1,-1.0,-1.0,400.0,10.19,-303.44,120.48
25704,2025-03-01 00:00:00+00:00,10.175663,13.333627,100.000000,72.000000,0.0,0.0,0.0,0.0,100.0,...,NaN,27.27,NaN,1,-1.0,-1.0,400.0,10.19,-303.44,120.48
5712,2025-03-01 00:00:00+00:00,15.349684,20.240198,39.289394,38.500687,0.0,0.0,0.0,0.0,100.0,...,NaN,27.27,NaN,1,-1.0,-1.0,400.0,10.19,-303.44,120.48
8568,2025-03-01 00:00:00+00:00,11.228318,15.088053,90.000000,63.000000,0.0,0.0,0.0,0.0,100.0,...,NaN,27.27,NaN,1,-1.0,-1.0,400.0,10.19,-303.44,120.48
22848,2025-03-01 00:00:00+00:00,16.299694,14.707222,120.529625,111.541046,0.0,0.0,0.0,0.0,100.0,...,NaN,27.27,NaN,1,-1.0,-1.0,400.0,10.19,-303.44,120.48


In [10]:
merged_df["hour"] = merged_df["time"].dt.hour
merged_df["weekday"] = merged_df["time"].dt.dayofweek

In [11]:
merged_df["activation_binary"] = (merged_df["LT_afrr_up_activ"] > 0).astype(int)

# Suggestions for Improvements
use wheather data from Latvia and Estonia